In [1]:
!pip install "pyiceberg[s3fs,hive]"

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/sharanya/.local/lib/python3.9/site-packages
sysconfig: /home/sharanya/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [2]:
! pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/sharanya/.local/lib/python3.9/site-packages
sysconfig: /home/sharanya/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [49]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)
tracetable=catalog.load_table("ingesttrace.traces") #mention tablename and namesapce

In [47]:
from kafka import KafkaConsumer
import json
try:
        # Create KafkaConsumer
        consumer = KafkaConsumer(
            "apmtraces",
            bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id="apmingestion",
            auto_offset_reset='earliest',
            consumer_timeout_ms=10000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile='./telemetry_zagaopenshift.pem',
            ssl_certfile='./telemetry_zagaopenshift.crt'
        )
        print("Kafka consumer created successfully.")
except Exception as e:
        print(f"Error creating Kafka consumer: {e}")

Kafka consumer created successfully.


In [48]:
import pyarrow as pa
from datetime import datetime
import json
def append_data(data):
  final={}
  traceId = ""
  route = ""
  method = ""
  status_code = ""

  resourceSpans = data.get('resourceSpans',[])
  print(resourceSpans)
  for resourceSpan in resourceSpans:
    for key in resourceSpan['resource']['attributes'] :
      if(key['key']=='service.name'):
        final.update({
            "servicename":key['value']['stringValue'],           
        })
      if(key['key']=='telemetry.sdk.language'):
        final.update({
            "language":key['value']['stringValue'],
            "tracedata":json.dumps(data)
        })
    for field in resourceSpan['scopeSpans']:  
      startTimeUnixNano = ((field['spans'][0].get('startTimeUnixNano')))
      for span in field['spans']:
        traceId = span.get('traceId')
        endTimeUnixNano = (span.get('endTimeUnixNano'))
        for attribute in span['attributes']:
          if attribute.get('key',{}) == "http.route":
              route = attribute.get('value',{}).get('stringValue')
          if attribute.get('key',{}) == "http.request.method":
              method = attribute.get('value',{}).get('stringValue')
          if attribute.get('key',{}) == "http.response.status_code":
              status_code = attribute.get('value',{}).get('intValue')
      createdTime = datetime.fromtimestamp(int(startTimeUnixNano)/ 1e9)
    duration = (int(endTimeUnixNano) - int(startTimeUnixNano))/1e9
  final.update({"createdTime": createdTime, 
                "traceId": traceId, 
                "startUnixNanoTime": int(startTimeUnixNano), 
                "endUnixNanoTime": int(endTimeUnixNano), 
                "duration": (duration), 
                "requestRoute": route, 
                "requestMethod": method, 
                "responseStatusCode": status_code})
  df=pa.Table.from_pylist([final])
  tracetable.append(df)
  print("Data appended successfully.")
  final={}
messages=[]
try:
  for message in consumer:
      data = json.loads(message.value.decode('utf-8'))
      append_data(data)
      messages.append(data)
      if(len(messages) > 5 ):
        consumer.close()
except Exception as e:
  print(f"Error consuming messages: {e}")
finally:
  consumer.close()
  print("Message consumption closed.")



ConsumerRecord(topic='apmtraces', partition=2, offset=41, timestamp=1721929069449, timestamp_type=0, key=None, value=b'{"resourceSpans":[{"resource":{"attributes":[{"key":"container.id","value":{"stringValue":"5d080326a974eed50a7af5282a1befec82ff0bce982cc13f1a65a74bea1f0f92"}},{"key":"host.arch","value":{"stringValue":"amd64"}},{"key":"host.name","value":{"stringValue":"order-srv-2-6467857f64-jj2wp"}},{"key":"k8s.container.name","value":{"stringValue":"order-srv-2"}},{"key":"k8s.deployment.name","value":{"stringValue":"order-srv-2"}},{"key":"k8s.namespace.name","value":{"stringValue":"observability-testcases"}},{"key":"k8s.node.name","value":{"stringValue":"zagaocp-worker1"}},{"key":"k8s.pod.name","value":{"stringValue":"order-srv-2-6467857f64-jj2wp"}},{"key":"k8s.replicaset.name","value":{"stringValue":"order-srv-2-6467857f64"}},{"key":"os.description","value":{"stringValue":"Linux 5.14.0-284.71.1.el9_2.x86_64"}},{"key":"os.type","value":{"stringValue":"linux"}},{"key":"process.comman